<a href="https://colab.research.google.com/github/MarianaDuartee/ProjetoFinal/blob/main/1_pubSub_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-pubsub
!pip install fsspec
!pip install gcsfs
!pip install apache_beam[gcp]
!pip install google-cloud-bigquery

In [ ]:
import csv
import time
import os

import fsspec
import gcsfs
import pandas as pd

import apache_beam as beam
from apache_beam import window
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam import coders

from google.cloud import pubsub_v1
from google.cloud import storage
from google.cloud import bigquery

# Configurando conta de serviço
service_account_key = r"/content/soulcode-projeto-final-4b88bea6e07a.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = service_account_key

Produtor

In [ ]:
# Setando o tópico de entrada (ingestão)
topico = 'projects/soulcode-projeto-final/topics/ingestor_dados'
publisher = pubsub_v1.PublisherClient()

entrada = r"/content/2_temp_temp_pandas_total_pop_ano_uf.csv"

# Visualizando entrada dos dados
with open(entrada, 'rb') as file:
    for row in file:
        print('Publicando no topico: ', topico)
        publisher.publish(topico,row)
        time.sleep(2)

Consumidor

In [ ]:
# Função de ACK (Aceite)
def monstrar_msg(mensagem):
  print(('Mensagem: {}'.format(mensagem)))
  mensagem.ack()

# Setando a subscrição de saída
subscription = 'projects/soulcode-projeto-final/subscriptions/consumidor_dados_violencia'
subscriber = pubsub_v1.SubscriberClient()

subscriber.subscribe(subscription,callback=monstrar_msg)

while True:
  time.sleep(2)

Template Stream

In [49]:
# Definindo a Pipeline
pipeline_options = {
    'project': 'soulcode-projeto-final' ,
    'runner': 'DataflowRunner',
    'region': 'southamerica-east1',
    'staging_location': 'gs://data_lake_ingest_data/temp',
    'temp_location': 'gs://data_lake_ingest_data/temp',
    'template_location': 'gs://data_lake_ingest_data/template/data_streaming',
    'save_main_session': True ,
    'streaming' : True }

pipeline_options = PipelineOptions.from_dictionary(pipeline_options)
p1 = beam.Pipeline(options=pipeline_options)
p1 = beam.Pipeline()

path = 'gs://data_lake_ingest_data/1_input/Taxa_analfabetismo.csv'

esquema = {str:'Unidade de Federação', 
           str:'Grupo de idade', 
           float:'2016', 
           float:'2017', 
           float:'2018', 
           float:'2019'}

# realizando a leitura e tratativa dos dados (se necessário)
mkt = (
    p1
    |"Extrair os dados" >> beam.io.ReadFromText(path, skip_header_lines=0, coder=coders.StrUtf8Coder())
    |"Window" >> beam.WindowInto(window.SlidingWindows(10,5))
    |'Separador' >> beam.Map(lambda record: record.split(','))
    |"Saida de dados" >> beam.FlatMap(print)
    |"Gravar resultado" >> beam.io.WriteToBigQuery(table='Teste_Beam_Apache',
                                                   dataset='Teste',
                                                   project='soulcode-projeto-final',
                                                   schema=esquema,
                                                   custom_gcs_temp_location='gs://data_lake_ingest_data/temp',
                                                   create_disposition=bigquery.CreateDisposition.CREATE_IF_NEEDED,
                                                   write_disposition=bigquery.WriteDisposition.WRITE_APPEND)
)

p1.run().wait_until_finish

/usr/local/lib/python3.7/dist-packages/apache_beam/io/gcp/bigquery.py:2103: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  is_streaming_pipeline = p.options.view_as(StandardOptions).streaming
/usr/local/lib/python3.7/dist-packages/apache_beam/io/gcp/bigquery_file_loads.py:1112: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


['\ufeff"Unidade da Federação"', '"Grupo de idade"', '"2016"', '"2017"', '"2018"', '"2019"']
['"Rondônia"', '"15 anos ou mais"', '"6.6"', '"7.2"', '"6.5"', '"6.4"']
['"Acre"', '"15 anos ou mais"', '"13.1"', '"12.1"', '"12.1"', '"11.7"']
['"Amazonas"', '"15 anos ou mais"', '"6.9"', '"6.1"', '"5.8"', '"5.4"']
['"Roraima"', '"15 anos ou mais"', '"6.6"', '"6.0"', '"6.0"', '"5.0"']
['"Pará"', '"15 anos ou mais"', '"9.2"', '"8.6"', '"8.8"', '"8.4"']
['"Amapá"', '"15 anos ou mais"', '"5.0"', '"5.0"', '"6.1"', '"5.5"']
['"Tocantins"', '"15 anos ou mais"', '"10.4"', '"10.2"', '"10.0"', '"9.7"']
['"Maranhão"', '"15 anos ou mais"', '"16.7"', '"16.8"', '"16.3"', '"15.6"']
['"Piauí"', '"15 anos ou mais"', '"17.2"', '"16.6"', '"16.6"', '"16.0"']
['"Ceará"', '"15 anos ou mais"', '"15.2"', '"14.2"', '"13.3"', '"13.6"']
['"Rio Grande do Norte"', '"15 anos ou mais"', '"14.7"', '"13.5"', '"12.9"', '"13.4"']
['"Paraíba"', '"15 anos ou mais"', '"16.3"', '"16.6"', '"16.1"', '"16.1"']
['"Pernambuco"', '"15 a

<bound method RunnerResult.wait_until_finish of <apache_beam.runners.portability.fn_api_runner.fn_runner.RunnerResult object at 0x7f1029a5e850>>

In [ ]:
pipeline_options = {
    'project': 'soulcode-projeto-final' ,
    'runner': 'DataflowRunner',
    'region': 'southamerica-east1',
    'staging_location': 'gs://data_lake_ingest_data/temporario',
    'temp_location': 'gs://data_lake_ingest_data/temporario',
    'template_location': 'gs://data_lake_ingest_data/template/data_streaming',
    'save_main_session': True ,
    'streaming' : True }

pipeline_options = PipelineOptions.from_dictionary(pipeline_options)

p1 = beam.Pipeline(options=pipeline_options)

subscription = 'projects/soulcode-projeto-final/subscriptions/consumidor_dados_violencia'
saida = 'projects/soulcode-projeto-final/topics/ingestor_dados_saida'

class separar_linhas(beam.DoFn):
  def process(self,record):
    return [record.decode("utf-8").split(',')]

#class filtro(beam.DoFn):
 # def process(self,record):
  #  if int(record[8]) > 0:
   #   return [record]

data = (
    p1
    |'Extrair os dados' >> beam.io.ReadFromText('/content/2_temp_temp_pandas_total_pop_ano_uf.csv', skip_header_lines=0)
    |'Separador' >> beam.Map(lambda record: record.split(','))
    | "Window" >> beam.WindowInto(window.SlidingWindows(10,5))
    #|'Saida de dados' >> beam.Map(print)
    "Escrever no Tópico" >> beam.io.WriteToPubSub(saida)
    #|'Gravar resultado' >> beam.io.WriteToText('resultado.txt')
)

result = p1.run()
result.wait_until_finish()